In [10]:
# === Cell 1: Versions & reset ===
import os, random, gc, sys, numpy as np, tensorflow as tf
print("Python:", sys.version)
print("TensorFlow:", tf.__version__)
print("Keras image_data_format:", tf.keras.backend.image_data_format())  # 'channels_last' 여야 정상

# 재현성 + 세션 초기화
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["TF_DETERMINISTIC_OPS"] = "1"
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
tf.keras.backend.clear_session(); gc.collect()

# GPU 확인
print("Physical GPUs:", tf.config.list_physical_devices('GPU'))


Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
TensorFlow: 2.19.0
Keras image_data_format: channels_last
Physical GPUs: []


In [11]:
# === Cell 2: Paths & constants ===
import os

DATA_ROOT = r"C:/Users/moont/Desktop/2025miniinterncode/lumbar_spinal_dataset"
TRAIN_DIR = os.path.join(DATA_ROOT, "training")
TEST_DIR  = os.path.join(DATA_ROOT, "testing")

CLASSES = ["Herniated_Disc", "No_Stenosis", "Thecal_Sac"]
IMG_SIZE = (200, 200)
INPUT_SHAPE = IMG_SIZE + (3,)   # 반드시 3채널
BATCH_SIZE = 32
VAL_SPLIT = 0.2

# EfficientNet 제외. 다음 중 선택:
BACKBONE = "resnet50"  # 'resnet50', 'densenet121', 'inceptionv3', 'mobilenetv2'

print("CLASSES:", CLASSES)
print("IMG_SIZE:", IMG_SIZE)
print("INPUT_SHAPE(should be 3ch):", INPUT_SHAPE)
assert INPUT_SHAPE[-1] == 3, "INPUT_SHAPE 마지막 채널이 3이 아닙니다!"


CLASSES: ['Herniated_Disc', 'No_Stenosis', 'Thecal_Sac']
IMG_SIZE: (200, 200)
INPUT_SHAPE(should be 3ch): (200, 200, 3)


In [12]:
# === Cell 3: Datasets (force RGB) ===
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, labels="inferred", label_mode="int", class_names=CLASSES,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    validation_split=VAL_SPLIT, subset="training",
    seed=SEED, shuffle=True, color_mode="rgb"
)
val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, labels="inferred", label_mode="int", class_names=CLASSES,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    validation_split=VAL_SPLIT, subset="validation",
    seed=SEED, shuffle=True, color_mode="rgb"
)
test_ds_raw = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR, labels="inferred", label_mode="int", class_names=CLASSES,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    shuffle=False, color_mode="rgb"
)

print(f"Batches -> Train:{len(train_ds_raw)}  Val:{len(val_ds_raw)}  Test:{len(test_ds_raw)}")

xb, yb = next(iter(train_ds_raw.take(1)))
print("첫 train 배치 shape:", xb.shape)  # (B, 200, 200, 3)
tf.debugging.assert_equal(tf.shape(xb)[-1], 3)


Found 4808 files belonging to 3 classes.
Using 3847 files for training.
Found 4808 files belonging to 3 classes.
Using 961 files for validation.
Found 1158 files belonging to 3 classes.
Batches -> Train:121  Val:31  Test:37
첫 train 배치 shape: (32, 200, 200, 3)


In [13]:
# === Cell 4: Preprocess/Augment ===
from tensorflow.keras import layers

AUTOTUNE = tf.data.AUTOTUNE
augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augment")
rescale = layers.Rescaling(1./255, name="rescale")

def prepare(ds, training=False):
    if training:
        ds = ds.map(lambda x, y: (augment(x, training=True), y), num_parallel_calls=AUTOTUNE)
    ds = ds.map(lambda x, y: (rescale(x), y), num_parallel_calls=AUTOTUNE)
    return ds.cache().prefetch(AUTOTUNE)

train_ds = prepare(train_ds_raw, training=True)
val_ds   = prepare(val_ds_raw,   training=False)
test_ds  = prepare(test_ds_raw,  training=False)

xb2, _ = next(iter(train_ds.take(1)))
print("전처리 후 배치 shape:", xb2.shape)  # (B, 200, 200, 3)
tf.debugging.assert_equal(tf.shape(xb2)[-1], 3)


전처리 후 배치 shape: (32, 200, 200, 3)


In [14]:
# === Cell 5: Backbone builder (no EfficientNet) ===
from tensorflow.keras import applications as KApps

def build_backbone_force_rgb(name: str):
    name = name.lower()
    inp = tf.keras.Input(shape=INPUT_SHAPE, name="force_rgb_input")  # 3채널 고정

    if name == "mobilenetv2":
        base = KApps.MobileNetV2(include_top=False, weights="imagenet", input_tensor=inp)
    elif name == "resnet50":
        base = KApps.ResNet50(include_top=False, weights="imagenet", input_tensor=inp)
    elif name == "densenet121":
        base = KApps.DenseNet121(include_top=False, weights="imagenet", input_tensor=inp)
    elif name == "inceptionv3":
        base = KApps.InceptionV3(include_top=False, weights="imagenet", input_tensor=inp)
    else:
        raise ValueError(f"Unsupported backbone for this notebook: {name}")

    return base


In [15]:
# === Cell 6: backbone build + full model (explicit head, short summary) ===
from tensorflow.keras import layers, models
from collections import deque

def short_summary(model, tail=15):
    """model.summary()의 마지막 tail줄만 출력"""
    buf = []
    model.summary(print_fn=lambda x: buf.append(x))
    for line in deque(buf, maxlen=tail):
        print(line)

def build_deep_model_explicit(backbone_name,
                              input_shape=(200, 200, 3),
                              dropout_rate=0.1):
    # 1) 백본 (이미지넷, 3채널 강제) — Cell 5의 build_backbone_force_rgb 사용
    base = build_backbone_force_rgb(backbone_name)
    base.trainable = False  # Stage 1: 백본 고정

    # 2) 입력/백본 통과
    inputs = base.inputs[0]          # 강제 3채널 input_tensor
    x = base.outputs[0]              # 백본 출력 feature

    # 3) 분류기 헤드 (명시적으로 단계별 구성)
    x = layers.GlobalAveragePooling2D(name="gap")(x)

    # Block 1
    x = layers.Dense(1024, activation="relu", name="dense_1")(x)
    x = layers.BatchNormalization(name="bn_1")(x)
    x = layers.Dropout(dropout_rate, name="dropout_1")(x)

    # Block 2
    x = layers.Dense(512, activation="relu", name="dense_2")(x)
    x = layers.BatchNormalization(name="bn_2")(x)
    x = layers.Dropout(dropout_rate, name="dropout_2")(x)

    # Block 3
    x = layers.Dense(512, activation="relu", name="dense_3")(x)
    x = layers.BatchNormalization(name="bn_3")(x)
    x = layers.Dropout(dropout_rate, name="dropout_3")(x)

    # Block 4
    x = layers.Dense(128, activation="relu", name="dense_4")(x)
    x = layers.BatchNormalization(name="bn_4")(x)
    x = layers.Dropout(dropout_rate, name="dropout_4")(x)

    # Output
    outputs = layers.Dense(len(CLASSES), activation="softmax", name="output")(x)

    model = models.Model(inputs, outputs, name=f"{backbone_name}_deep_model")
    return model, base

# —— 모델 생성 & 짧은 summary 출력 ——
tf.keras.backend.clear_session()
model, base = build_deep_model_explicit(
    backbone_name=BACKBONE,        # 예: "mobilenetv2"
    input_shape=INPUT_SHAPE,
    dropout_rate=0.4
)

print("\n===== ✅ Backbone summary (short) =====")
short_summary(base, tail=15)

print("\n===== ✅ Full model summary (short) =====")
short_summary(model, tail=15)



===== ✅ Backbone summary (short) =====


Model: "resnet50"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ force_rgb_input     │ (None, 200, 200,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 206, 206,  │          0 │ force_rgb_input[… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 100, 100,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 100, 100,  │        256 │ c

Model: "resnet50_deep_model"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ force_rgb_input     │ (None, 200, 200,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 206, 206,  │          0 │ force_rgb_input[… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 100, 100,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 100, 100,  │    

In [16]:
# === Cell 8: two-stage training (EarlyStopping 적용) ===
from tensorflow.keras.callbacks import EarlyStopping

def train_two_stage(model, base, train_ds, val_ds,
                    epochs_stage1=30, epochs_stage2=20,
                    finetune_unfreeze_top_k=20,
                    lr_stage1=1e-3, lr_stage2=1e-4,
                    patience=5, monitor="val_loss"):

    early_stop = EarlyStopping(
        monitor=monitor, patience=patience, restore_best_weights=True, verbose=1
    )

    # Stage 1 — 백본 고정
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_stage1),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    print(f"\n[Stage 1] up to {epochs_stage1} epochs (monitor: {monitor}, patience={patience})")
    hist1 = model.fit(
        train_ds, validation_data=val_ds,
        epochs=epochs_stage1, callbacks=[early_stop], verbose=1
    )

    # Stage 2 — 백본 일부 파인튜닝
    for layer in base.layers[-finetune_unfreeze_top_k:]:
        layer.trainable = True

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_stage2),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    print(f"\n[Stage 2] unfreeze top {finetune_unfreeze_top_k} layers, up to {epochs_stage2} epochs")
    hist2 = model.fit(
        train_ds, validation_data=val_ds,
        epochs=epochs_stage2, callbacks=[early_stop], verbose=1
    )
    return hist1, hist2


In [17]:
# === Cell 9: evaluation (정확도/혼동행렬/민감도·특이도/리포트/ROC-AUC) ===
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np

def evaluate_full(model, test_ds, class_names):
    # 예측
    y_true, y_prob = [], []
    for xb, yb in test_ds:
        preds = model.predict(xb, verbose=0)
        y_prob.append(preds)
        y_true.extend(yb.numpy())
    y_true = np.array(y_true)
    y_prob = np.vstack(y_prob)
    y_pred = np.argmax(y_prob, axis=1)

    # 정확도
    acc = accuracy_score(y_true, y_pred)
    print(f"\n✅ 전체 정확도: {acc:.4f}")

    # 혼동행렬
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))
    print("\n[혼동 행렬] (rows=true, cols=pred)\n", cm)

    # 분류 리포트
    print("\n[분류 리포트: Precision / Recall / F1]")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

    # 민감도(Sensitivity), 특이도(Specificity)
    TP = np.diag(cm)
    FN = cm.sum(axis=1) - TP
    FP = cm.sum(axis=0) - TP
    TN = cm.sum() - (TP + FP + FN)
    sensitivity = TP / (TP + FN + 1e-8)
    specificity = TN / (TN + FP + 1e-8)
    print("\n[클래스별 민감도 / 특이도]")
    for i, cls in enumerate(class_names):
        print(f"{cls:15s} | 민감도 {sensitivity[i]:.4f} | 특이도 {specificity[i]:.4f}")

    # ROC-AUC (One-vs-Rest)
    y_true_bin = label_binarize(y_true, classes=list(range(len(class_names))))
    auc_macro = roc_auc_score(y_true_bin, y_prob, multi_class="ovr", average="macro")
    auc_weighted = roc_auc_score(y_true_bin, y_prob, multi_class="ovr", average="weighted")
    print(f"\n[ROC-AUC] macro={auc_macro:.4f} | weighted={auc_weighted:.4f}")


In [18]:
# === Cell 10: train & evaluate (에폭 확대 + 얼리스타핑) ===
_ = train_two_stage(
    model, base,
    train_ds=train_ds, val_ds=val_ds,
    epochs_stage1=30, epochs_stage2=20,     # 반복 횟수 확대
    finetune_unfreeze_top_k=20,             # 상위 20개 레이어 파인튜닝
    lr_stage1=1e-3, lr_stage2=1e-4,
    patience=5, monitor="val_loss"          # 얼리스타핑 기준/인내심
)

evaluate_full(model, test_ds, class_names=CLASSES)



[Stage 1] up to 30 epochs (monitor: val_loss, patience=5)
Epoch 1/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.3449 - loss: 1.5948 - val_accuracy: 0.3684 - val_loss: 1.0969
Epoch 2/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.3522 - loss: 1.3166 - val_accuracy: 0.3746 - val_loss: 1.1411
Epoch 3/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - accuracy: 0.3787 - loss: 1.2171 - val_accuracy: 0.3788 - val_loss: 1.1137
Epoch 4/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.3936 - loss: 1.1604 - val_accuracy: 0.4121 - val_loss: 1.0730
Epoch 5/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.4034 - loss: 1.1311 - val_accuracy: 0.4037 - val_loss: 1.0966
Epoch 6/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.4024 - loss: 1.1141 - val_accuracy: 0.4225 - val_loss: 1.0521
Epoch 7/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.4081 - loss: 1.1039 - val_accuracy: 0.4266 - val_loss: 1.0486
Epoch 8/30
121/121 ━━━━━━━━━━━━━━━━━━━